In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Common Model Algorithms
from catboost import CatBoostClassifier
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from sklearn.linear_model import Lasso, LogisticRegression, LinearRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.naive_bayes import MultinomialNB
import sklearn.ensemble as ensemble
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from scipy.stats import zscore

########## FEATURE SELECTION
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix, classification_report, matthews_corrcoef, cohen_kappa_score, accuracy_score, average_precision_score, roc_auc_score

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# split into train and test
from sklearn.model_selection import train_test_split

# missing value
from missingpy import MissForest

import warnings
warnings.filterwarnings("ignore")

In [5]:
data_raw2 = pd.read_csv("./train_transaction.csv")
data_val2 = pd.read_csv("./test_transaction.csv")

In [6]:
tr = data_raw2.copy(deep=True)
te = data_val2.copy(deep=True)

## Removing outliers, imputing nulls, and encoding catogorical data

In [7]:
import warnings
warnings.filterwarnings("ignore")
data_cleaner = [tr,te]
for i,dataset in enumerate(data_cleaner):
    # null imputation
    for c in dataset.columns:
        if c == "isFraud" or c == "TransactionID":
            continue
        if dataset[c].dtype == 'int64' or dataset[c].dtype == 'float64':
            Mu = dataset[c].mean()
            St = dataset[c].std()
            dataset[c][dataset[c] > (Mu + 2.5*St)] = Mu
            dataset[c][dataset[c] < (Mu - 2.5*St)] = Mu
            #dataset[c].interpolate(method="spline", order = 1, inplace=True)
            dataset[c].fillna(dataset[c].median() , inplace=True)

        else:
            dataset[c].fillna(dataset[c].mode()[0] , inplace=True)
# enumerating categorical data
label = LabelEncoder()
for dataset in data_cleaner:
    delcols = []
    print(dataset.head())
    for c in dataset.columns:
        if dataset[c].dtype == 'object':
            dataset[c+'_code'] = label.fit_transform(dataset[c])
            delcols += [c]
    for c in delcols:
        dataset.drop(c , axis=1, inplace=True)
    print(dataset.head())
    print('-'*15)


   TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   
3        2987003        0          86499            50.0         W  18132   
4        2987004        0          86506            50.0         H   4497   

   card2  card3       card4  card5  ... V330  V331  V332  V333  V334 V335  \
0  361.0  150.0    discover  142.0  ...  0.0   0.0   0.0   0.0   0.0  0.0   
1  404.0  150.0  mastercard  102.0  ...  0.0   0.0   0.0   0.0   0.0  0.0   
2  490.0  150.0        visa  166.0  ...  0.0   0.0   0.0   0.0   0.0  0.0   
3  567.0  150.0  mastercard  117.0  ...  0.0   0.0   0.0   0.0   0.0  0.0   
4  514.0  150.0  mastercard  102.0  ...  0.0   0.0   0.0   0.0   0.0  0.0   

  V336  V337  V338  V339  
0  0.0   0.0   0.0   0.0  
1  0.0   0.0   0.0  

## Dealing with imbalanced dataset
I chose to downsample the non-fradulant transactions and did this 4 time to create 4 different  groups of training data.

In [8]:
# train data
TR = []
TRl = []
TS = []
TSl = []
#tr = pd.concat([labels,tr],axis=1)
for i in range(4):
    fraud = tr[tr.isFraud == 1]
    nonfraud = tr[tr.isFraud == 0]
    print(fraud.shape , nonfraud.shape)
    numsample = int(np.floor(fraud.shape[0]*1))
    fraudsamp = fraud.sample(n=numsample)
    nonfraudsamp = nonfraud.sample(n=1*numsample)
    Train = pd.concat([fraudsamp,nonfraudsamp],ignore_index=True)
    Train = Train.sample(frac=1).reset_index(drop=True)
    print(Train.shape)
    y = Train.isFraud
    x = Train.copy(deep=True)
    x.drop(["isFraud"],axis = 1,inplace=True)

    # split Train into train and test
    x_tr, x_tst, y_tr, y_tst = train_test_split(x,y,test_size=0.3,random_state=42)
    TR.append(x_tr)
    TRl.append(y_tr)
    TS.append(x_tst)
    TSl.append(y_tst)

(20663, 394) (569877, 394)
(41326, 394)
(20663, 394) (569877, 394)
(41326, 394)
(20663, 394) (569877, 394)
(41326, 394)
(20663, 394) (569877, 394)
(41326, 394)


## Feature Selection
I decided to use Genetic Algorithm because it's a supervised guided search, but a huge downside of employing this algorithm is its computational cost. I have also brought a sample script of using Boruta feature selection method at the end of this document in the appendix section.

In [13]:
# Genetic Algorithm
import warnings
warnings.filterwarnings("ignore")
def initpop(n,numpop,tot):
    # n: length of each chromosome
    # numpop: size of the population
    # tot: number of available features
    pop = []
    for i in range(numpop):
        pop += [tuple(np.random.permutation(tot)[:n])]
    return pop
def breed(POP,tot,p=0.75):
    Pop = []
    for crom in POP:
        c = tuple([])
        for i in range(len(crom)):
            P = np.random.rand()
            if P < p:
                c += (crom[i],)
        for i in range(len(crom)-len(c)):
                options = list( set(range(tot)).difference(set(c)) )
                val = options[np.random.randint(len(options))]
                c += (val,)
        Pop += [c]
    return Pop
def Eval(pop,x_tr,y_tr):
    ac = []
    X, Xt, Y, Yt = train_test_split(x_tr,y_tr,test_size=0.3,random_state=42)
    for crom in pop:
        mdl = RandomForestClassifier()
        #mdl = XGBClassifier()
        mdl.fit(X.iloc[:,list(crom)], Y)
        ac += [roc_auc_score(Yt,mdl.predict(Xt.iloc[:,list(crom)]))]
    return ac
def GA(pop,tot,x_tr,y_tr,max_iter=100):
    ### Main loop
    pops = []
    pops.append(pop)
    evals = []
    elite = ()
    prob = 0.8
    for i in range(max_iter):
        evals.append(Eval(pops[i],x_tr,y_tr))
        if elite == ():
            M = 0
            Ind = 0
        Mflag = 0
        m = 100
        ind = 0
        for j,e in enumerate(evals[i]):
            if e > M:
                Mflag = 1
                M = e
                Ind = j
            if e < m:
                m = e
                ind = j
        if elite != ():
            pops[i][ind] = elite
        if Mflag:
            elite = pops[i][Ind]
            print("Progress made at iteration",i)
            print(M)
        if i != max_iter-1:
            if np.mean(evals[i]) > 0.8: # if the population is good enough, don't change them a lot
                prob = 0.75
            else:
                prob = 0.25
            pops.append(breed(pops[i],tot,prob))
    return elite,M

Features = [[] for i in range(len(TR))]
Accs = [0 for i in range(len(TR))]
for i,x_tr in enumerate(TR):
    for n in range(50,66,5):
        print("*"*10,"n =",n)
        pop = initpop(n,10,len(x_tr.columns))
        features,performance = GA(pop,len(x_tr.columns),x_tr,TRl[i],300)
        print(performance)
        if performance > Accs[i]:
            Features[i] = list(features)
            Accs[i] = performance
            print("n =",n,"is chosen for training group",i)

********** n = 50
Progress made at iteration 0
0.8046731500648411
Progress made at iteration 2
0.820439936783947
Progress made at iteration 58
0.8205141774106697
Progress made at iteration 92
0.8241766353383259
Progress made at iteration 219
0.8255788432269502
0.8255788432269502
n = 50 is chosen for training group 0
********** n = 55
Progress made at iteration 0
0.7983272758363222
Progress made at iteration 1
0.803158970529976
Progress made at iteration 3
0.8051965783461652
Progress made at iteration 7
0.8058982531679718
Progress made at iteration 13
0.8066963664576812
Progress made at iteration 14
0.8154977308283407
Progress made at iteration 44
0.8241597214330247
Progress made at iteration 115
0.8284509942827283
0.8284509942827283
n = 55 is chosen for training group 0
********** n = 60
Progress made at iteration 0
0.7917888433012968
Progress made at iteration 1
0.8045402816470585
Progress made at iteration 6
0.8106439179508312
Progress made at iteration 18
0.8171420969843861
Progress

In [14]:
print(Accs,'\n')

[0.8359386235001853, 0.8261335341783467, 0.8218951670729959, 0.8259490111463437] 



As you can can see, working on the 1st training group is best in the sense that it leads to a group of features that can better classify the group.

## Testing some models
using TR[0] (the 1st group), I'm going to look at the performance of some well-known models.

In [15]:
n = 0
cols = TR[n].columns[Features[n]]
x_tr = TR[n]
y_tr = TRl[n]
x_tst = TS[n]
y_tst = TSl[n]
len(cols)

60

In [ ]:
from sklearn.model_selection import cross_validate, ShuffleSplit, GridSearchCV
from sklearn.linear_model import LogisticRegressionCV, SGDClassifier
from sklearn import ensemble, naive_bayes, svm, tree, discriminant_analysis, neighbors, feature_selection
MLA = [    
        # Generalized Linear Models
        LogisticRegressionCV(),
    
        # SVM
        svm.SVC(probability = True),
        svm.LinearSVC(),
    
        # KNN
        neighbors.KNeighborsClassifier(weights='distance'),
    
        #Discriminant Analysis
        discriminant_analysis.LinearDiscriminantAnalysis(),
        discriminant_analysis.QuadraticDiscriminantAnalysis(),
     
        # Naive Bayes
        naive_bayes.BernoulliNB(),
        naive_bayes.GaussianNB(),
    
        #Trees    
        tree.DecisionTreeClassifier(),
    
        # Ensemble Methods
        ensemble.AdaBoostClassifier(),
        ensemble.BaggingClassifier(),
        ensemble.ExtraTreesClassifier(),
        ensemble.GradientBoostingClassifier(),
        ensemble.RandomForestClassifier()
     
    ]

cv_split = ShuffleSplit(n_splits = 4, test_size = .3, train_size = .7, random_state = 0)
MLA_columns = ['MLA Name','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean','MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

row_index = 0
for i,alg in enumerate(MLA):
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    cv_results = cross_validate(alg, x_tr.iloc[:,Features[n]],y_tr, cv  = cv_split, scoring='roc_auc', return_train_score=True)
    
    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
        
    row_index+=1
    print(i,"out of",len(MLA))

MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)

0 out of 14
1 out of 14
2 out of 14
3 out of 14
4 out of 14
5 out of 14
6 out of 14
7 out of 14
8 out of 14
9 out of 14
10 out of 14
11 out of 14


In [17]:
MLA_compare

,MLA Name,MLA Train Accuracy Mean,MLA Test Accuracy Mean,MLA Time
11,ExtraTreesClassifier,1,0.900122,0.262989
13,RandomForestClassifier,0.999593,0.897416,0.258548
10,BaggingClassifier,0.999491,0.892388,1.23092
12,GradientBoostingClassifier,0.87923,0.868544,2.48929
9,AdaBoostClassifier,0.854159,0.847522,1.00456
4,LinearDiscriminantAnalysis,0.81129,0.805916,0.159469
5,QuadraticDiscriminantAnalysis,0.785122,0.779432,0.0718229
8,DecisionTreeClassifier,1,0.772463,0.180463
6,BernoulliNB,0.756542,0.751736,0.0466278
7,GaussianNB,0.68863,0.687355,0.0285662


## Play with some famous models in more detail

## LightGBM

In [18]:
import lightgbm as lgbm

In [22]:
d_tr = lgbm.Dataset(x_tr.iloc[:,Features[n]],y_tr)
params = {}
params['learning_rate']= 0.6
params['boosting_type']='gbdt' # gbdt, rf, dart, goss
params['objective']='binary'
params['metric']='auc'
params['sub_feature']=0.5
params['num_leaves']= 120
params['min_data']=50
params['max_depth']=9
params['tree_learner'] = 'serial' #serial, feature, data, voting
params['lambad_l2'] = 1e-2
lgb= lgbm.train(params, d_tr, 1500) # set numiters to 10,000

In [24]:
ac_2 = roc_auc_score(y_tst,np.floor(np.array(lgb.predict(x_tst.iloc[:,Features[n]]))+0.5))
print(ac_2)

0.8524790208035934


## Tuning XGB

In [25]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'booster':["gbtree", "gblinear"],
    'learning_rate': [0.03],
    'n_estimators': [900],
    'max_depth': [12,14,15,16,18],
    'min_child_weight': [0],
    'gamma': [0]
}
# Create a based model
xgb = XGBClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgb, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

# Fit the random search model
grid_search.fit(x_tr.iloc[:,Features[n]],y_tr)
print(grid_search.best_params_)

ac_2 = roc_auc_score(y_tst,grid_search.predict(x_tst.iloc[:,Features[n]]))
print(ac_2)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  5.9min finished


{'booster': 'gbtree', 'gamma': 0, 'learning_rate': 0.03, 'max_depth': 14, 'min_child_weight': 0, 'n_estimators': 900}
0.8631190317623857


## Tuning RandomForestClassifier

## randomized search

In [27]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 600, stop = 1200, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(90, 140, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [28]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(x_tr.iloc[:,Features[n]],y_tr)
print(rf_random.best_params_)

ac_2 = roc_auc_score(y_tst,rf_random.predict(x_tst.iloc[:,Features[n]]))
print(ac_2)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 18.8min finished


{'n_estimators': 666, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 134, 'bootstrap': False}
0.8587626307680167


# grid search

In [31]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [110,120,130,140],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1],
    'min_samples_split': [2,3,4,5],
    'n_estimators': [600,700,800]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

# Fit the random search model
grid_search.fit(x_tr.iloc[:,features],y_tr)
print(grid_search.best_params_)

ac_2 = roc_auc_score(y_tst,grid_search.predict(x_tst.iloc[:,features]))
print(ac_2)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed: 10.2min finished


{'bootstrap': False, 'max_depth': 120, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 800}
0.8522293564478358


## Validating tuned models

In [35]:
# Random Forest Classifier
rf = RandomForestClassifier(bootstrap=False, max_depth=120, max_features='sqrt',\
                            min_samples_leaf=1, min_samples_split=3, n_estimators=800)
rf.fit(x_tr[cols], y_tr)

ac_2 = roc_auc_score(y_tst,rf.predict(x_tst[cols]))
print(ac_2)

0.8598905182505321


In [36]:
# XGBoost Classifier
xgbc = XGBClassifier(
 learning_rate =0.03,
 n_estimators=900,
 max_depth=14,
 min_child_weight=0,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27,
 eval_metric='auc')
xgbc = xgbc.fit(x_tr[cols],y_tr)
ac_2 = roc_auc_score(y_tst,xgbc.predict(x_tst[cols]))
print(ac_2)

0.8681195272396979


In [39]:
# CatBoost Classifier (as the number of parameters is small, 
# I tuned it here by running this cell multiple times with different values)
cbc = CatBoostClassifier(
 iterations=50,
 learning_rate =0.5,
 depth=16,
 l2_leaf_reg=20,
 loss_function='Logloss')
cbc = cbc.fit(x_tr[cols],y_tr)
ac_2 = roc_auc_score(y_tst,cbc.predict(x_tst[cols]))
print(ac_2)

0:	learn: 0.5366292	total: 2.43s	remaining: 1m 58s
1:	learn: 0.4859901	total: 4.92s	remaining: 1m 58s
2:	learn: 0.4589838	total: 7.38s	remaining: 1m 55s
3:	learn: 0.4391421	total: 9.84s	remaining: 1m 53s
4:	learn: 0.4304051	total: 12.3s	remaining: 1m 50s
5:	learn: 0.4165138	total: 14.8s	remaining: 1m 48s
6:	learn: 0.3909541	total: 17.3s	remaining: 1m 46s
7:	learn: 0.3842132	total: 19.8s	remaining: 1m 43s
8:	learn: 0.3782804	total: 22.2s	remaining: 1m 41s
9:	learn: 0.3628113	total: 24.7s	remaining: 1m 38s
10:	learn: 0.3555952	total: 25.8s	remaining: 1m 31s
11:	learn: 0.3480688	total: 28.1s	remaining: 1m 28s
12:	learn: 0.3383553	total: 30.5s	remaining: 1m 26s
13:	learn: 0.3347627	total: 32.9s	remaining: 1m 24s
14:	learn: 0.3309992	total: 35.4s	remaining: 1m 22s
15:	learn: 0.3258499	total: 37.8s	remaining: 1m 20s
16:	learn: 0.3170130	total: 40.2s	remaining: 1m 18s
17:	learn: 0.3107438	total: 42.8s	remaining: 1m 16s
18:	learn: 0.3033789	total: 45.3s	remaining: 1m 13s
19:	learn: 0.2969812	t

In [40]:
# Extra Trees Classifier (same as catboost tuning)
etc = ensemble.ExtraTreesClassifier(bootstrap=False, max_depth=40, max_features=40,\
                            min_samples_leaf=1, min_samples_split=2, n_estimators=100)
etc.fit(x_tr[cols], y_tr)

ac_2 = roc_auc_score(y_tst,etc.predict(x_tst[cols]))
print(ac_2)

0.8522293564478358


In [41]:
# Bagging Classifier (same as catboost tuning)
bc = ensemble.BaggingClassifier(bootstrap=False,max_features=40,n_estimators=85)
bc.fit(x_tr[cols], y_tr)

ac_2 = roc_auc_score(y_tst,bc.predict(x_tst[cols]))
print(ac_2)

0.8614264979184748


## Ensemble

In [42]:
## Naive Majority Voting Ensemble
agg = xgbc.predict(x_tst[cols]) + rf.predict(x_tst[cols]) + bc.predict(x_tst[cols]) \
+ etc.predict(x_tst[cols]) + cbc.predict(x_tst[cols])
th = 3
agg[agg<th] = 0
agg[agg>=th] = 1
ac2 = roc_auc_score(y_tst,agg)
ac2

0.8664244951908514

In [43]:
entr = np.array( [rf.predict(x_tr[cols]),bc.predict(x_tr[cols]),etc.predict(x_tr[cols]),cbc.predict(x_tr[cols])] )
ents = np.array( [rf.predict(x_tst[cols]),bc.predict(x_tst[cols]),etc.predict(x_tst[cols]),cbc.predict(x_tst[cols])] )

In [44]:
# XGBoost Ensemble
xgbc = XGBClassifier( learning_rate =0.03,
 n_estimators=900,
 max_depth=14,
 min_child_weight=0,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27,
 eval_metric='auc')
xgbc = xgbc.fit(entr.T,y_tr)
ac_2 = roc_auc_score(y_tst,xgbc.predict(ents.T))
print(ac_2)

0.8598905182505321


In [45]:
## ANN Ensemble
# Keras ANN

# Initialising the ANN
ann = Sequential()

# Adding the input layer and the first hidden layer
ann.add(Dense(units = 50, kernel_initializer = 'uniform', activation = 'relu', input_dim = 4))
for i in range(1):
    # Adding the hidden layers
    ann.add(Dense(units = 50, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer (1 output layer because it's a classification problem)
ann.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN (Performing stochastic gradient descendent to the ANN)
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [46]:
ann.fit(entr.T, y_tr, batch_size = 10, epochs = 50, validation_data = (ents.T, y_tst), shuffle=False)


Train on 28928 samples, validate on 12398 samples
Epoch 1/50
28928/28928 [==============================] - 6s 217us/step - loss: 0.0243 - accuracy: 0.9962 - val_loss: 1.6679 - val_accuracy: 0.8622
Epoch 2/50
28928/28928 [==============================] - 6s 208us/step - loss: 4.8432e-07 - accuracy: 1.0000 - val_loss: 1.9422 - val_accuracy: 0.8622
Epoch 3/50
28928/28928 [==============================] - 6s 210us/step - loss: 6.5882e-08 - accuracy: 1.0000 - val_loss: 2.1441 - val_accuracy: 0.8622
Epoch 4/50
28928/28928 [==============================] - 6s 209us/step - loss: 1.4037e-08 - accuracy: 1.0000 - val_loss: 2.3161 - val_accuracy: 0.8622
Epoch 5/50
28928/28928 [==============================] - 6s 206us/step - loss: 3.8818e-09 - accuracy: 1.0000 - val_loss: 2.4656 - val_accuracy: 0.8622
Epoch 6/50
28928/28928 [==============================] - 6s 204us/step - loss: 1.3559e-09 - accuracy: 1.0000 - val_loss: 2.5947 - val_accuracy: 0.8622
Epoch 7/50
28928/28928 [=================

In [47]:
ac_2 = roc_auc_score(y_tst,ann.predict(ents.T))
print(ac_2)

0.8837716138965981


In [55]:
# SVM Ensemble
svc_lvl2_optimal = {'C': 0.1,
              'kernel': 'linear',
              'cache_size': 100, 
              'shrinking': True,
              'decision_function_shape' : 'ovo'}

svc_lvl2 = svm.SVC(**svc_lvl2_optimal)
svc_lvl2.fit(entr.T, y_tr)
svc2_train_score = svc_lvl2.score(entr.T, y_tr)
ac_2 = roc_auc_score(y_tst,svc_lvl2.predict(ents.T))
print(svc2_train_score,ac_2)

1.0 0.8647179089441774


As shown, ANN ensemble works best and results in 2 percent improvement compare to the best single model performance (XGBoost)

## Final Stage - Single Performance

In [214]:
# rf
rf = RandomForestClassifier(bootstrap=False, max_depth=150, max_features=50,\
                            min_samples_leaf=1, min_samples_split=3, n_estimators=900)
rf.fit(x[cols], y)
labs = pd.DataFrame(rf.predict(te[cols]),columns=["isFraud"])
out = pd.concat([te["TransactionID"],labs],axis=1)
out.to_csv("submission.csv",index=False)

In [30]:
# xgb
xgbc = XGBClassifier(learning_rate =0.03, n_estimators=900, max_depth=14, min_child_weight=0,\
                     gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',\
                     nthread=4, scale_pos_weight=1, eval_metric='auc')
xgbc = xgbc.fit(x_tr[cols],y_tr)
labs = pd.DataFrame(xgbc.predict(te[cols]),columns=["isFraud"])
out = pd.concat([te["TransactionID"],labs],axis=1)
out.to_csv("submission.csv",index=False)

In [ ]:
# etc
etc = ensemble.ExtraTreesClassifier(bootstrap=False, max_depth=50, max_features=15,\
                            min_samples_leaf=1, min_samples_split=2, n_estimators=100)
etc.fit(x[cols], y)
labs = pd.DataFrame(etc.predict(te[cols]),columns=["isFraud"])
out = pd.concat([te["TransactionID"],labs],axis=1)
out.to_csv("submission.csv",index=False)

In [ ]:
# bc
bc = ensemble.BaggingClassifier(bootstrap=False,max_features=30,n_estimators=15)
bc.fit(x[cols], y)
labs = pd.DataFrame(bc.predict(te[cols]),columns=["isFraud"])
out = pd.concat([te["TransactionID"],labs],axis=1)
out.to_csv("submission.csv",index=False)

## Ape

In [73]:
# Boruta
from boruta import BorutaPy
group = []

# Random Forests for Boruta 
#rf_boruta = RandomForestClassifier(bootstrap = True, n_estimators = 300, max_depth = 100, max_features = 3, min_samples_leaf=3, min_samples_split=8, n_jobs=-1, random_state=0)

for n in range(55,91,5):
    rf_boruta = RandomForestClassifier(n_estimators = 300, max_depth = n, n_jobs=-1, random_state=0)
    # Perform Boruta
    boruta = BorutaPy(rf_boruta, n_estimators='auto', verbose=2, alpha=0.03, max_iter=300, perc=75)
    boruta.fit(x_tr.values, y_tr.values.ravel())
    group += [boruta.support_]

Iteration: 	1 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	2 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	3 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	4 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	5 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	6 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	7 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	8 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	9 / 300
Confirmed: 	73
Tentative: 	114
Rejected: 	206
Iteration: 	10 / 300
Confirmed: 	73
Tentative: 	114
Rejected: 	206
Iteration: 	11 / 300
Confirmed: 	73
Tentative: 	114
Rejected: 	206
Iteration: 	12 / 300
Confirmed: 	73
Tentative: 	114
Rejected: 	206
Iteration: 	13 / 300
Confirmed: 	76
Tentative: 	88
Rejected: 	229
Iteration: 	14 / 300
Confirmed: 	76
Tentative: 	88
Rejected: 	229
Iteration: 	15 / 300
Confirmed: 	76
Tentative: 	88
Rejected: 	229
Iteration: 	16 / 300
Confirmed:

Iteration: 	126 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	127 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	128 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	129 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	130 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	131 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	132 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	133 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	134 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	135 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	136 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	137 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	138 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	139 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 	303
Iteration: 	140 / 300
Confirmed: 	79
Tentative: 	11
Rejected: 

Iteration: 	250 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	251 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	252 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	253 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	254 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	255 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	256 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	257 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	258 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	259 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	260 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	261 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	262 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	263 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration: 	264 / 300
Confirmed: 	79
Tentative: 	8
Rejected: 	306
Iteration:

Iteration: 	74 / 300
Confirmed: 	79
Tentative: 	18
Rejected: 	296
Iteration: 	75 / 300
Confirmed: 	79
Tentative: 	18
Rejected: 	296
Iteration: 	76 / 300
Confirmed: 	80
Tentative: 	17
Rejected: 	296
Iteration: 	77 / 300
Confirmed: 	80
Tentative: 	17
Rejected: 	296
Iteration: 	78 / 300
Confirmed: 	80
Tentative: 	17
Rejected: 	296
Iteration: 	79 / 300
Confirmed: 	80
Tentative: 	17
Rejected: 	296
Iteration: 	80 / 300
Confirmed: 	80
Tentative: 	17
Rejected: 	296
Iteration: 	81 / 300
Confirmed: 	80
Tentative: 	17
Rejected: 	296
Iteration: 	82 / 300
Confirmed: 	80
Tentative: 	16
Rejected: 	297
Iteration: 	83 / 300
Confirmed: 	80
Tentative: 	16
Rejected: 	297
Iteration: 	84 / 300
Confirmed: 	80
Tentative: 	16
Rejected: 	297
Iteration: 	85 / 300
Confirmed: 	80
Tentative: 	16
Rejected: 	297
Iteration: 	86 / 300
Confirmed: 	80
Tentative: 	16
Rejected: 	297
Iteration: 	87 / 300
Confirmed: 	80
Tentative: 	15
Rejected: 	298
Iteration: 	88 / 300
Confirmed: 	80
Tentative: 	15
Rejected: 	298
Iteration:

Iteration: 	197 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	198 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	199 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	200 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	201 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	202 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	203 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	204 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	205 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	206 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	207 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	208 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	209 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	210 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 	301
Iteration: 	211 / 300
Confirmed: 	80
Tentative: 	12
Rejected: 

...

Iteration: 	4 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	5 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	6 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	7 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	8 / 300
Confirmed: 	0
Tentative: 	393
Rejected: 	0
Iteration: 	9 / 300
Confirmed: 	66
Tentative: 	129
Rejected: 	198
Iteration: 	10 / 300
Confirmed: 	66
Tentative: 	129
Rejected: 	198
Iteration: 	11 / 300
Confirmed: 	66
Tentative: 	129
Rejected: 	198
Iteration: 	12 / 300
Confirmed: 	66
Tentative: 	129
Rejected: 	198
Iteration: 	13 / 300
Confirmed: 	69
Tentative: 	93
Rejected: 	231
Iteration: 	14 / 300
Confirmed: 	69
Tentative: 	93
Rejected: 	231
Iteration: 	15 / 300
Confirmed: 	69
Tentative: 	93
Rejected: 	231
Iteration: 	16 / 300
Confirmed: 	69
Tentative: 	93
Rejected: 	231
Iteration: 	17 / 300
Confirmed: 	70
Tentative: 	80
Rejected: 	243
Iteration: 	18 / 300
Confirmed: 	70
Tentative: 	80
Rejected: 	243
Iteration: 	19 / 300
C

Iteration: 	128 / 300
Confirmed: 	73
Tentative: 	17
Rejected: 	303
Iteration: 	129 / 300
Confirmed: 	73
Tentative: 	17
Rejected: 	303
Iteration: 	130 / 300
Confirmed: 	73
Tentative: 	17
Rejected: 	303
Iteration: 	131 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 	304
Iteration: 	132 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 	304
Iteration: 	133 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 	304
Iteration: 	134 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 	304
Iteration: 	135 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 	304
Iteration: 	136 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 	304
Iteration: 	137 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 	304
Iteration: 	138 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 	304
Iteration: 	139 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 	304
Iteration: 	140 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 	304
Iteration: 	141 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 	304
Iteration: 	142 / 300
Confirmed: 	73
Tentative: 	16
Rejected: 

Iteration: 	251 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	252 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	253 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	254 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	255 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	256 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	257 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	258 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	259 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	260 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	261 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	262 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	263 / 300
Confirmed: 	73
Tentative: 	9
Rejected: 	311
Iteration: 	264 / 300
Confirmed: 	73
Tentative: 	9
Rejected: 	311
Iteration: 	265 / 300
Confirmed: 	73
Tentative: 	9
Rejected: 	31

Iteration: 	75 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	76 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	77 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	78 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	79 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	80 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	81 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	82 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	83 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	84 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	85 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	86 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	87 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	88 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration: 	89 / 300
Confirmed: 	72
Tentative: 	20
Rejected: 	301
Iteration:

Iteration: 	198 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	199 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	200 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	201 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	202 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	203 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	204 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	205 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	206 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	207 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	208 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	209 / 300
Confirmed: 	73
Tentative: 	10
Rejected: 	310
Iteration: 	210 / 300
Confirmed: 	73
Tentative: 	9
Rejected: 	311
Iteration: 	211 / 300
Confirmed: 	73
Tentative: 	9
Rejected: 	311
Iteration: 	212 / 300
Confirmed: 	73
Tentative: 	9
Rejected: 	31